In [1]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(ggplot2, warn.conflicts = FALSE)
library(readr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(sf, warn.conflicts = FALSE)

options(repr.plot.width = 9, repr.plot.res = 300)


source("src/paths/paths.R")
path.piemonte <- file.path(path.ds, "ARPA", "PIEMONTE")
path.dwnl <- file.path(path.piemonte, "site-data")


Linking to GEOS 3.11.0, GDAL 3.5.3, PROJ 9.1.0; sf_use_s2() is TRUE



In [2]:
open_dataset(path.dwnl, format = "parquet") |>
    select(date = data, T_MAX = tmax, T_MIN = tmin, tclasse, identifier = id_punto) |>
    mutate(tclasse = cast(tclasse, utf8()), identifier = cast(identifier, utf8())) |>
    map_batches(
        ~ pivot_longer(. |> collect(), cols = c(T_MAX, T_MIN), names_to = "variable", values_to = "value") |>
            as_record_batch()
    ) |>
    arrange(variable, identifier, date) |>
    # write_parquet("test.parquet")
    write_dataset(file.path(path.piemonte, "dataset"), format = "feather", partitioning = "variable")


In [ ]:
open_dataset(file.path(path.piemonte, "dataset"), format = "feather") |>
    filter(variable == "T_MIN") |>
    collect()


In [11]:
library(sf, warn.conflicts = FALSE)
read_csv_arrow(file.path(path.piemonte, "metadata.csv"), col_select = c(data_inizio_sensore, data_fine_sensore, quota_da_pc, quota_stazione, esposizione, tipo_staz, sigla_prov, comune, denominazione, indirizzo_localita, identifier = id_punto_misura, lon = longitudine_e_wgs84_d, lat = latitudine_n_wgs84_d)) |>
    st_as_sf(coords = c("longitudine_e_wgs84_d", "latitudine_n_wgs84_d"), crs = "WGS84") |>
    write_sf(file.path(path.piemonte, "metadata.geojson"))


In [31]:
ds <- open_dataset(file.path(path.piemonte, "dataset"), format = "feather")


In [2]:
metadata <- read_sf(file.path(path.piemonte, "metadata.geojson"))


In [30]:
metadata |> group_by(id_punto_misura) |> filter(n() > 1)

,data_inizio_sensore,data_fine_sensore,quota_da_pc,quota_stazione,esposizione,tipo_staz,sigla_prov,comune,denominazione,indirizzo_localita,id_punto_misura,geometry
,<date>,<date>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<POINT [arc_degree]>
1,2006-09-28,2018-04-17,0.00,1410,ENE,PTV,TO,BALME,BALME,FRAZIONE CORNETTI,PIE-001019-901,POINT (7.22194 45.3025)
2,2018-04-18,NA,0.00,1410,ENE,PTV,TO,BALME,BALME,FRAZIONE CORNETTI,PIE-001019-901,POINT (7.22194 45.3025)
3,2005-09-06,2006-03-24,0.00,2045,WNW,BHNPTV,TO,BARDONECCHIA,BARDONECCHIA PRANUDIN,PRANUDIN,PIE-001022-905,POINT (6.70803 45.10627)
4,2006-03-25,NA,0.00,2045,WNW,BHNPTV,TO,BARDONECCHIA,BARDONECCHIA PRANUDIN,PRANUDIN,PIE-001022-905,POINT (6.70803 45.10627)
5,1999-09-07,2018-08-08,0.00,1581,ENE,NPT,TO,CERESOLE REALE,CERESOLE VILLA,VILLA,PIE-001073-901,POINT (7.21083 45.44028)
6,2018-08-09,NA,0.00,1581,ENE,NPT,TO,CERESOLE REALE,CERESOLE VILLA,VILLA,PIE-001073-901,POINT (7.21083 45.44028)
7,2001-10-01,2008-05-30,0.00,1149,NE,IT,TO,OULX,BEAULARD DORA DI BARDONECCHIA,PONTE S.S. 335,PIE-001175-902,POINT (6.75806 45.04667)
8,2009-06-25,NA,0.00,1149,NE,IT,TO,OULX,BEAULARD DORA DI BARDONECCHIA,PONTE S.S. 335,PIE-001175-902,POINT (6.75806 45.04667)
9,2001-10-01,2001-12-08,0.00,260,ESE,IPT,TO,PARELLA,PARELLA CHIUSELLA,PONTE S.P. 63,PIE-001179-900,POINT (7.79806 45.42)


In [3]:
metadata |>
    group_by(id_punto_misura) |>
    filter(n() > 1) |>
    select(denominazione, id_punto_misura, geometry, quota_stazione)


Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



denominazione,id_punto_misura,geometry,quota_stazione
<chr>,<chr>,<POINT [arc_degree]>,<int>
BALME,PIE-001019-901,POINT (7.22194 45.3025),1410
BALME,PIE-001019-901,POINT (7.22194 45.3025),1410
BARDONECCHIA PRANUDIN,PIE-001022-905,POINT (6.70803 45.10627),2045
BARDONECCHIA PRANUDIN,PIE-001022-905,POINT (6.70803 45.10627),2045
CERESOLE VILLA,PIE-001073-901,POINT (7.21083 45.44028),1581
CERESOLE VILLA,PIE-001073-901,POINT (7.21083 45.44028),1581
BEAULARD DORA DI BARDONECCHIA,PIE-001175-902,POINT (6.75806 45.04667),1149
BEAULARD DORA DI BARDONECCHIA,PIE-001175-902,POINT (6.75806 45.04667),1149
PARELLA CHIUSELLA,PIE-001179-900,POINT (7.79806 45.42),260


In [26]:
metadata |>
    arrange(data_inizio_sensore) |>
    group_by(id_punto_misura) |>
    slice_tail() |>
    rename(identifier = id_punto_misura, elevation = quota_stazione, anagrafica = denominazione) |>
    write_sf(file.path(path.piemonte, "metadata_deduped.geojson"))
